In [1]:
import shutil
import os
import re
import pandas as pd

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
root_path = 'gdrive/MyDrive/Explainable_Wound_Classification/'

In [4]:
#load the label probabilities for the images
labels_df = pd.read_csv(root_path + 'Cropped_Images_Wound_Stage_Probabilities.csv', index_col='Image')
labels_df.head()

,hemostasis,inflammatory,proliferative,maturation
Image,,,,
Day 8_A8-4-L.png,0.181818,0.090909,0.545455,0.181818
Day 4_A8-3-R.png,0.090909,0.909091,0.000000,0.000000
Day 14_Y8-4-L.png,0.000000,0.000000,0.090909,0.909091
Day 7_Y8-4-L.png,0.000000,0.000000,0.454545,0.545455
Day 2_A8-1-L.png,0.181818,0.727273,0.090909,0.000000


In [5]:
#get properties of images from name

props = labels_df.index.map(lambda x: re.match('^Day (\d+)_(Y|A)8-(\d)-(L|R)', x).groups())

labels_df['Day'] = props.map(lambda x: int(x[0]))
labels_df['Age'] = props.map(lambda x: x[1])
labels_df['Mouse'] = props.map(lambda x: int(x[2]))
labels_df['Side'] = props.map(lambda x: x[3])

labels_df.head()

,hemostasis,inflammatory,proliferative,maturation,Day,Age,Mouse,Side
Image,,,,,,,,
Day 8_A8-4-L.png,0.181818,0.090909,0.545455,0.181818,8,A,4,L
Day 4_A8-3-R.png,0.090909,0.909091,0.000000,0.000000,4,A,3,R
Day 14_Y8-4-L.png,0.000000,0.000000,0.090909,0.909091,14,Y,4,L
Day 7_Y8-4-L.png,0.000000,0.000000,0.454545,0.545455,7,Y,4,L
Day 2_A8-1-L.png,0.181818,0.727273,0.090909,0.000000,2,A,1,L


In [6]:
#separate images into train, val, and test
# val = Y8-4-R and A8-1-L images
# test = Y8-4-L and A8-1-R images

os.makedirs(root_path + 'Split_images/train/', exist_ok=True)
os.makedirs(root_path + 'Split_images/val/', exist_ok=True)
os.makedirs(root_path + 'Split_images/test/', exist_ok=True)

for index, row in labels_df.iterrows():
    if row['Age'] == 'Y' and row['Mouse'] == 4 and row['Side'] == 'L' or row['Age'] == 'A' and row['Mouse'] == 1 and row['Side'] == 'R':
        shutil.copy(root_path + 'Circle_Cropped_images/' + index, root_path + 'Split_images/test/')
    elif row['Age'] == 'Y' and row['Mouse'] == 4 and row['Side'] == 'R' or row['Age'] == 'A' and row['Mouse'] == 1 and row['Side'] == 'L':
        shutil.copy(root_path + 'Circle_Cropped_images/' + index, root_path + 'Split_images/val/')
    else:
        shutil.copy(root_path + 'Circle_Cropped_images/' + index, root_path + 'Split_images/train/')